In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # Libreria numpy
import pandas as pd # procesamiento de data, archivos CSV I/O (e.g. pd.read_csv)

# Los archivos de entrada están disponibles en el directorio "../input/".
# Por ejemplo, listado de los archivos en el directorio input

import os
print(os.listdir("../input"))

# Cualquier resultado que se escriba en el presente directorio será guardado como salida.
%matplotlib inline

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf, pdb
import PIL
import matplotlib.image as mpimg

In [ ]:
WEIGHTS_INIT_STDEV = .1

def net(image):
    conv1 = _conv_layer(image, 32, 9, 1)
    conv2 = _conv_layer(conv1, 64, 3, 2)
    conv3 = _conv_layer(conv2, 128, 3, 2)
    resid1 = _residual_block(conv3, 3)
    resid2 = _residual_block(resid1, 3)
    resid3 = _residual_block(resid2, 3)
    resid4 = _residual_block(resid3, 3)
    resid5 = _residual_block(resid4, 3)
    conv_t1 = _conv_tranpose_layer(resid5, 64, 3, 2)
    conv_t2 = _conv_tranpose_layer(conv_t1, 32, 3, 2)
    conv_t3 = _conv_layer(conv_t2, 3, 9, 1, relu=False)
    preds = tf.nn.tanh(conv_t3) * 150 + 255./2
    return preds

def _conv_layer(net, num_filters, filter_size, strides, relu=True):
    weights_init = _conv_init_vars(net, num_filters, filter_size)
    strides_shape = [1, strides, strides, 1]
    net = tf.nn.conv2d(net, weights_init, strides_shape, padding='SAME')
    net = _instance_norm(net)
    if relu:
        net = tf.nn.relu(net)

    return net

def _conv_tranpose_layer(net, num_filters, filter_size, strides):
    weights_init = _conv_init_vars(net, num_filters, filter_size, transpose=True)

    batch_size, rows, cols, in_channels = [i.value for i in net.get_shape()]
    new_rows, new_cols = int(rows * strides), int(cols * strides)
    # new_shape = #tf.pack([tf.shape(net)[0], new_rows, new_cols, num_filters])

    new_shape = [batch_size, new_rows, new_cols, num_filters]
    tf_shape = tf.stack(new_shape)
    strides_shape = [1,strides,strides,1]

    net = tf.nn.conv2d_transpose(net, weights_init, tf_shape, strides_shape, padding='SAME')
    net = _instance_norm(net)
    return tf.nn.relu(net)

def _residual_block(net, filter_size=3):
    tmp = _conv_layer(net, 128, filter_size, 1)
    return net + _conv_layer(tmp, 128, filter_size, 1, relu=False)

def _instance_norm(net, train=True):
    batch, rows, cols, channels = [i.value for i in net.get_shape()]
    var_shape = [channels]
    mu, sigma_sq = tf.nn.moments(net, [1,2], keep_dims=True)
    shift = tf.Variable(tf.zeros(var_shape))
    scale = tf.Variable(tf.ones(var_shape))
    epsilon = 1e-3
    normalized = (net-mu)/(sigma_sq + epsilon)**(.5)
    return scale * normalized + shift

def _conv_init_vars(net, out_channels, filter_size, transpose=False):
    _, rows, cols, in_channels = [i.value for i in net.get_shape()]
    if not transpose:
        weights_shape = [filter_size, filter_size, in_channels, out_channels]
    else:
        weights_shape = [filter_size, filter_size, out_channels, in_channels]

    weights_init = tf.Variable(tf.truncated_normal(weights_shape, stddev=WEIGHTS_INIT_STDEV, seed=1), dtype=tf.float32)
    return weights_init

In [ ]:
def save_image(image, file_path):
    '''
    Guarda una imagen como un archivo jpg file. La imagen es dada como 
    un array numpy con valores de píxeles entre 0 y 255.
    
    :param image:
        El array numpy de la imagen.
        type: ndarray
    :param file_path:
        La ruta completa para guardar la imagen mezclada, 
        es decir, image path + image name
        type: str
    :return:
        Guardar la imagen con un archivo jpeg.
    '''
    
    # Asegurar que los valores de píxeles están entre 0 y 255.
    image = np.clip(image, 0.0, 255.0)
    
    # Convertir a bytes.
    image = image.astype(np.uint8)
    
    # Escribir el archivo imagen en el formato jpeg.
    with open(file_path, 'wb') as file:
        PIL.Image.fromarray(image).save(file, 'jpeg')

In [ ]:
def feed_forward(image_path, output_path, checkpoint_dir, style_path=None):
    '''
    Como ya tenemos un punto de control pre-entrenado, podemos usarlo 
    para generar una imagen mixta.
    
    :param image_path:
        La ruta y el nombre del archivo que va ser transferido.
        type: str
    :param style_path:
        La ruta y nombre de archivo de la imagen estilo.
        Esto es solo para mostrar y no tiene nada que ver con el 
        procesamiento.
        type: str
    :param output_path:
        La ruta para almacenar la imagen mixta, incluyendo su nombre de archivo.
        type: str
    :param checkpoint_dir:
        La ruta y nombre de archivo del punto de control pre-entrenado.
        type: str
    :return:
        Guarda la imagen mixta y la muestra.
    '''
    
    # Construye un graph y una sesión.
    with tf.Graph().as_default(), tf.Session() as sess:
        # Lee la imagen contenido desde un archivo como un array numpy.
        content_image = mpimg.imread(image_path)
        
        # Lee la imagen estilo desde un archivo si se proporciona.
        if style_path is not None:
            style_image = mpimg.imread(style_path)
        
        # Dado que la red de transformación de imagen requiere una matriz 4-D,
        # Se tendrá que expandir una dimensión en el eje = 0.
        content_image = np.expand_dims(content_image, axis=0)
        
        # Define un marcador de posición 4-D para la imagen.
        image_holder = tf.placeholder(
            tf.float32, content_image.shape, 'input_image')
        
        # Deja que la imagen fluya a través de la red de transformación.
        output_image = net(image_holder)
        
        # Restaura el punto de control pre-entrenado.
        saver = tf.train.Saver()
        saver.restore(sess, checkpoint_dir)
        
        # Ejecuta la sesión.
        feed_dict = {image_holder: content_image}
        mixed_image = sess.run(output_image, feed_dict)
        
        # Guarda la imagen mixta.
        with open(output_path, 'wb') as file:
            save_image(mixed_image[0], output_path)
        
        # Si la imagen estilo es proporcionada, mostrar la imagen contenido, mixta y estilo.
        if style_path is not None:
            fig, axes = plt.subplots(1, 3, figsize=(20, 20))
            
        # Si la imagen estilo no es proporcionada, moestrar la imagen contenido y mixta.
        else:
            fig, axes = plt.subplots(1, 2, figsize=(20, 20))
        
        # Usa la interpolación para suavizar los píxeles.
        smooth = True

        # Tipo de interpolación.
        if smooth:
            interpolation = 'sinc'
        else:
            interpolation = 'nearest'
            
        # Grafica la imagen contenido.
        # Notar que los valores de los píxeles son normalizados al rango
        # [0.0, 1.0] dividiendo con 255.
        ax = axes.flat[0]
        ax.imshow(content_image[0] / 255.0, interpolation=interpolation)
        ax.set_xlabel('Contenido')

        # Grafica la imagen mixta.
        ax = axes.flat[1]
        ax.imshow(mixed_image[0] / 255.0, interpolation=interpolation)
        ax.set_xlabel('Mixta')
        
        if style_path is not None:
            # Grafica la imagen estilo
            ax = axes.flat[2]
            ax.imshow(style_image / 255.0, interpolation=interpolation)
            ax.set_xlabel("Estilo")
        
        # Elimina las marcas de todos los gráficos.
        for ax in axes.flat:
            ax.set_xticks([])
            ax.set_yticks([])

        # Asegura de que el gráfico se muestra correctamente con múltiples gráficos
        # en una celda simple del Notebook.
        plt.show()

In [ ]:
INPUT_PATH = '../input/facultad1.jpg'
OUTPUT_PATH = 'facultad1.jpg'
CHECKPOINT_DIR = '../input/la_muse.ckpt'
STYLE_PATH = '../input/la_muse.jpg' # Opcional. Solo para mostrarlo.

# Ejecuta el código.
feed_forward(INPUT_PATH, OUTPUT_PATH, CHECKPOINT_DIR, STYLE_PATH)